# Example notebook for hydrateflash
Date: August 13, 2017  
Author: Kristopher N. Darnell  

This notebook is as an example of using the hydratefalsh software.  
Over the coming months, this project may grow.

## Components

* The elementary part of this code is a thermodynamic basis for computation, which consists of:  
  1. composition in mole fraction
  2. pressure in bar  
  3. temperature in Kelvin
* At present, pressure and temperature take fixed units of bar and Kelvin, respectively.
* Composition is defined as a list and then converted into instances of specific components. 
* See example below.

In [1]:
import component_properties as comp_table

comps = ['h2o', 'n2', 'co2']
comp_list = [comp_table.Component(ii) for ii in comps]
for comp in comp_list:
    print('{0:3s}: Tc={1:5.1f} K, Pc={2:5.1f} bar, MW={3:3.1f} g/mol'.format(
            comp.compname, comp.Tc, comp.Pc, comp.MW))

h2o: Tc=647.3 K, Pc=220.5 bar, MW=18.0 g/mol
n2 : Tc=126.2 K, Pc= 34.0 bar, MW=28.0 g/mol
co2: Tc=304.2 K, Pc= 73.8 bar, MW=44.0 g/mol


* The specific name convention for the components is not fixed.  
* You are able to specify components using a variety of aliases for each component.  
* See example below

In [2]:
comps = ['water', 'n_2', 'carbon dioxide']
comp_list = [comp_table.Component(ii) for ii in comps]
for comp in comp_list:
    print('{0:3s}: Tc={1:5.1f} K, Pc={2:5.1f} bar, MW={3:3.1f} g/mol'.format(
            comp.compname, comp.Tc, comp.Pc, comp.MW))

h2o: Tc=647.3 K, Pc=220.5 bar, MW=18.0 g/mol
n2 : Tc=126.2 K, Pc= 34.0 bar, MW=28.0 g/mol
co2: Tc=304.2 K, Pc= 73.8 bar, MW=44.0 g/mol


* The complete list of methods and attributes for each components is quite vast. 
* See 'component_properties.py' for additional details

## Fugacities

* There are fugacity objectes for various phases including:
  1. Liquid/vapor hydrocarbons phases
  2. aqueous phase
  3. hydrate phase (structure 1 and structure 2)
* Each phase fugacity has a file associated with it.
* See example below

In [3]:
import vlhc_srk_eos as hc
import aq_hb_eos as aq
import h_vdwpm_eos as hyd
import numpy as np

# Composition, pressure, temperature
x = np.ones(len(comps)) / len(comps)  # equal amounts of each component
P = 30  # bar
T = 300  # Kelvin

# Create instances of each equation of state
SRK_obj = hc.SrkEos(comp_list, T, P)
Aq_obj = aq.HegBromEos(comp_list, T, P)

# Access the fugacity of each component in each phase
hc_fug = SRK_obj.calc(comp_list, T, P, x, phase='vapor')
aq_fug = Aq_obj.calc(comp_list, T, P, x)
for ii, comp in enumerate(comp_list):
    print('fugacity of {0} in the hydrocarbon phase is {1:3.2f} bar'.format(
            comp.compname, hc_fug[ii]))
    print('fugacity of {0} in the aqueous phase is {1:3.2f} bar \n'.format(
            comp.compname, aq_fug[ii]))

fugacity of h2o in the hydrocarbon phase is 7.06 bar
fugacity of h2o in the aqueous phase is 0.02 bar 

fugacity of n2 in the hydrocarbon phase is 11.13 bar
fugacity of n2 in the aqueous phase is 90942.10 bar 

fugacity of co2 in the hydrocarbon phase is 8.36 bar
fugacity of co2 in the aqueous phase is 87.82 bar 



## Flash

* The main purpose of hydrateflash is to calculate the amount of each component in each phase.
  * This is called a "flash" calculation.
* The flash calculation for aqueous and hydrocarbon phases is complex, but tractable.
* The flash calculation when hydrate is a possible phase is far more complex.
* At present, CSMGem is the best option for performing hydrate flash calculations.
* hydrateflash performs flash calculations fast with complete transparency. 
* hydrateflash has been validated against CSMGem

In [4]:
import flashalgorithm as fc

flash = fc.FlashController(components=['water', 'methane'])
output = flash.main_handler(
            compobjs=flash.compobjs, 
            z=np.asarray([0.8, 0.2]), 
            T=280.0, 
            P=70.0)
stable_dict = {phase: ii for ii, (phase, alpha) in 
                 enumerate(zip(flash.phases, flash.alpha_calc)) 
                 if alpha > 1e-10}
print('Calculation considers the following phases:\n{0}\n'.format(flash.phases))
print('The stable phases are:')
for phase, index in stable_dict.items():
    print('\n{0}: {1:3.4f} mol.%'.format(phase, flash.alpha_calc[index]))
    for ii, comp in enumerate(flash.compobjs): 
        print('\twith {0:3.4f} mol.% {1}'.format(
                flash.x_calc[ii, index],
                comp.compname))



Calculation considers the following phases:
['aqueous', 'vapor', 'lhc', 's1']

The stable phases are:

vapor: 0.0678 mol.%
	with 0.0002 mol.% h2o
	with 0.9998 mol.% ch4

s1: 0.9322 mol.%
	with 0.8582 mol.% h2o
	with 0.1418 mol.% ch4


In [5]:
flash2 = fc.FlashController(components=['ethane', 'methane'])

In [6]:
output2 = flash2.main_handler(
            compobjs=flash2.compobjs, 
            z=np.asarray([0.5, 0.5]), 
            T=260.0, 
            P=50.0)
stable_dict = {phase: ii for ii, (phase, alpha) in 
                 enumerate(zip(flash2.phases, flash2.alpha_calc)) 
                 if alpha > 1e-10}
print('Calculation considers the following phases:\n{0}\n'.format(flash2.phases))
print('The stable phases are:')
for phase, index in stable_dict.items():
    print('\n{0}: {1:3.4f} mol.%'.format(phase, flash2.alpha_calc[index]))
    for ii, comp in enumerate(flash2.compobjs): 
        print('\twith {0:3.4f} mol.% {1}'.format(
                flash2.x_calc[ii, index],
                comp.compname))

Calculation considers the following phases:
['vapor', 'lhc']

The stable phases are:

vapor: 0.7731 mol.%
	with 0.4437 mol.% c2h6
	with 0.5563 mol.% ch4

lhc: 0.2269 mol.%
	with 0.6919 mol.% c2h6
	with 0.3081 mol.% ch4


## More to come...stay tuned!